<a href="https://colab.research.google.com/github/Axel02leon/Intro-to-Machine-Learning-/blob/main/Homework_7_Problem_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
torch.cuda.is_available()
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time
from torchvision import datasets
import datetime


In [ ]:
# Data Preparation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += identity
        out = self.relu(out)
        return out

In [ ]:
class ResNet10(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet10, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels),
            )
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
     out = self.conv1(x)
     out = self.bn1(out)  # Apply BatchNorm to the output of conv1
     out = self.relu(out)  # Apply ReLU activation
     out = self.layer1(out)
     out = self.layer2(out)
     out = self.layer3(out)
     out = self.layer4(out)
     out = self.avg_pool(out)
     out = torch.flatten(out, 1)  # Flatten for the fully connected layer
     out = self.fc(out)
     return out


In [ ]:

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available. Using GPU.")
    device = torch.device("cuda")
else:
    print("CUDA is not available. Using CPU.")
    device = torch.device("cpu")

CUDA is available. Using GPU.


In [ ]:
from torch.amp import GradScaler, autocast  # Updated import
def resnet10(num_classes=10):
    return ResNet10(ResidualBlock, [2, 2, 2, 2], num_classes=num_classes)

def train_model(model, criterion, optimizer, train_loader, val_loader, n_epochs=200):
    model.to(device)
    train_losses = []
    start_time = time.time()

    # Initialize GradScaler for mixed-precision training
    scaler = GradScaler("cuda")

    for epoch in range(1, n_epochs + 1):
        model.train()
        running_loss = 0.0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()

            # Use autocast for mixed-precision
            with autocast("cuda"):
                outputs = model(inputs)
                loss = criterion(outputs, targets)

            # Scale the loss for mixed-precision, backward pass, and optimizer step
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        if epoch % 10 == 0 or epoch == 1 or epoch == n_epochs:
            print(f"Epoch {epoch}/{n_epochs}, Training Loss: {avg_train_loss:.4f}")

    end_time = time.time()
    training_time = end_time - start_time
    print(f"Total Training Time: {training_time:.2f} seconds")
    return train_losses, training_time



In [ ]:
# Evaluation Function
def evaluate_model(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)
            correct += (predictions == targets).sum().item()
            total += targets.size(0)

    accuracy = correct / total * 100
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy

# Initialize Model, Loss, and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet10(num_classes=10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# Train the Model
print("Training ResNet-10...")
train_losses, training_time = train_model(model, criterion, optimizer, train_loader, test_loader, n_epochs=200)

# Evaluate the Model
print("\nEvaluating ResNet-10...")
test_accuracy = evaluate_model(model, test_loader)

Training ResNet-10...
Epoch 1/200, Training Loss: 1.4565
Epoch 10/200, Training Loss: 0.3379
Epoch 20/200, Training Loss: 0.1842
Epoch 30/200, Training Loss: 0.1127
Epoch 40/200, Training Loss: 0.0729
Epoch 50/200, Training Loss: 0.0532
Epoch 60/200, Training Loss: 0.0425
Epoch 70/200, Training Loss: 0.0306
Epoch 80/200, Training Loss: 0.0274
Epoch 90/200, Training Loss: 0.0261
Epoch 100/200, Training Loss: 0.0270
Epoch 110/200, Training Loss: 0.0216
Epoch 120/200, Training Loss: 0.0221
Epoch 130/200, Training Loss: 0.0321
Epoch 140/200, Training Loss: 0.0291
Epoch 150/200, Training Loss: 0.0297
Epoch 160/200, Training Loss: 0.0320
Epoch 170/200, Training Loss: 0.0267
Epoch 180/200, Training Loss: 0.0252
Epoch 190/200, Training Loss: 0.0293
Epoch 200/200, Training Loss: 0.0248
Total Training Time: 6936.12 seconds

Evaluating ResNet-10...
Test Accuracy: 91.32%
